In [1]:
import joblib
import re
import pandas as pd
import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt

In [2]:
df = pd.read_excel("./df_sample06.xlsx", engine="openpyxl", header=0, index_col=0)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 0 to 75
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   내용           76 non-null     object
 1   학교           76 non-null     object
 2   school_code  76 non-null     int64 
 3   전공           76 non-null     object
 4   진로           76 non-null     object
 5   career_code  76 non-null     int64 
 6   인성           76 non-null     object
dtypes: int64(2), object(5)
memory usage: 4.8+ KB


In [4]:
df['전공'].unique()

array(['중국어', '없음', '영어', '프랑스어', '일본어', '러시아어', '스페인어', '독일어'],
      dtype=object)

In [5]:
major_code = {'없음':0, '영어':1, '중국어':2, '일본어':3, '독일어':4, '러시아어':5, '스페인어': 6, '프랑스어': 7} 

In [6]:
major_code_list = []
major = df['전공']
for i in major:
    major_code_list.append(major_code[i])
df['major_code'] = major_code_list
df.columns.tolist()
df = df[['내용', '학교', 'school_code', '전공', 'major_code', '진로', 'career_code', '인성']]

In [7]:
df['인성'].unique()

array(['노인', '조별과제', '동아리', '교육', '임원', '장애인', '저소득층'], dtype=object)

In [8]:
personal_code = {'임원' : 0, '동아리':1, '조별과제' : 2, '교육' : 3, '노인' :4, '장애인': 5, '저소득층': 6}

In [9]:
personal_code_list = []
personal = df['인성']
for i in personal:
    personal_code_list.append(personal_code[i])
df['personal_code'] = personal_code_list

In [10]:
df.to_excel("./df_sample07.xlsx")

In [11]:
content = df['내용']

In [12]:
content.info()

<class 'pandas.core.series.Series'>
Int64Index: 76 entries, 0 to 75
Series name: 내용
Non-Null Count  Dtype 
--------------  ----- 
76 non-null     object
dtypes: object(1)
memory usage: 1.2+ KB


In [13]:
context = pd.DataFrame()

In [14]:
tagger = Okt()

for i in range(len(content)):
    text = content[i]
    text.lower()
    texts = text.split(". ")
    for text in texts:
        if len(text) <= 5:
            continue
        text = re.sub(r"[^가-힣a-zA-Z0-9 ]", "", text)
        temp_df = pd.DataFrame({0:[text], 1:[i]})
        context = pd.concat([context, temp_df], ignore_index = True)

In [15]:
context.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1869 entries, 0 to 1868
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1869 non-null   object
 1   1       1869 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 29.3+ KB


In [16]:
def get_morphs(text):
    _morphs = tagger.morphs(text)
    _morphs = [word for word in _morphs if len(word) > 1]
    return _morphs

In [17]:
def get_nouns(text):
    _nouns = tagger.nouns(text)
    _nouns = [word for word in _nouns if len(word) > 1]
    return _nouns

In [18]:
tfidf = TfidfVectorizer(tokenizer=get_morphs)

In [19]:
tdm = tfidf.fit_transform(context[0])

In [20]:
tfidf.get_feature_names_out()[-10:]

array(['힘들었', '힘들었고', '힘들었다', '힘들었던', '힘들었지만', '힘들지만', '힘써야', '힘쓰고',
       '힘쓰고자', '힘쓸'], dtype=object)

In [21]:
def get_similarity(idx, tdm):
    axis_0, aixs_1 = tdm.shape
    if 0 < idx < axis_0-1:
        sim_1 = cosine_similarity(tdm[idx], tdm[:idx])
        sim_2 = cosine_similarity(tdm[idx], tdm[idx+1:])
        sim = np.concatenate((sim_1, sim_2), axis=1)
        
    elif idx == 0:
        sim = cosine_similarity(tdm[0], tdm[1:])
    elif idx == axis_0 - 1:
        sim = cosine_similarity(tdm[idx], tdm[:idx])
    
    sim_idx = np.argmax(sim)
    similarity = sim.max()
    if sim_idx >= idx :
        sim_idx += 1

    return sim_idx, similarity

In [22]:
idx = 412
sim_idx, similarity = get_similarity(idx, tdm)
print(sim_idx)
print("-"*20)
print(context.loc[idx, 0])
print("-"*20)
print(context.loc[sim_idx, 0])
print("-"*20)
print(similarity)

417
--------------------
특히 2학년까지 외교관을 꿈꿔왔던 만큼 이한탁 씨 양현정씨와 같이 해외에서 누명을 쓴 한국인들의 안타까운 상황과 외교부 대응의 한계에 주목하였고 직접 외국법 자문사로서 전문 로펌을 설립하여 현지 법이나 언어에 능숙하지 못한 우리 국민들을 돕겠다고 결심하였다
--------------------
외국법 자문사의 꿈을 이룬다면 각국의 법률 전문가들을 모아 로펌을 만들어 국민들에게 실질적인 도움을 주는 것이 내 꿈이다
--------------------
0.20250966766944728


In [23]:
idx = 69
sim_idx, similarity = get_similarity(idx, tdm)
print(sim_idx)
print("-"*20)
print(context.loc[idx, 0])
print("-"*20)
print(context.loc[sim_idx, 0])
print("-"*20)
print(similarity)

1776
--------------------
영어토론동아리에서는 친구들과 다양한 분야의 주제를 선정하여 넓은 시야를 가지고 영어토론을 할 수 있게 되었다
--------------------
또 3학년 때는 영어토론 동아리에 참여해 친구들과 다양한 주제에 대해 토론했다
--------------------
0.5032652453651283


In [24]:
idx = 125
sim_idx, similarity = get_similarity(idx, tdm)
print(sim_idx)
print("-"*20)
print(context.loc[idx, 0])
print("-"*20)
print(context.loc[sim_idx, 0])
print("-"*20)
print(similarity)

1020
--------------------
2학년 때 저소득층 아이들 교육 봉사를 했다
--------------------
2학년 때 신문에서 저소득층 아이들에게 수업을 해주는 봉사활동 광고를 보고 참여해보았다
--------------------
0.39600657002843587


In [25]:
with open('Coverletter.pkl', 'wb') as f:
    joblib.dump({'tdm': tdm, 'context': context}, f)

In [26]:
context.to_excel("context.xlsx")